<a href="https://colab.research.google.com/github/Anvarka/RecSys/blob/master/RECSYS_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://voudy-data.s3.eu-north-1.amazonaws.com/wsdm_data.tar.gz
!tar -xf /content/wsdm_data.tar.gz
!mkdir data
!mv -t data members.csv song_extra_info.csv songs.csv train.csv
!pip install catboost

--2021-11-27 23:18:30--  https://voudy-data.s3.eu-north-1.amazonaws.com/wsdm_data.tar.gz
Resolving voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)... 52.95.170.32
Connecting to voudy-data.s3.eu-north-1.amazonaws.com (voudy-data.s3.eu-north-1.amazonaws.com)|52.95.170.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 613394184 (585M) [application/x-gzip]
Saving to: ‘wsdm_data.tar.gz.4’

wsdm_data.tar.gz.4  100%[===================>] 584.98M  23.2MB/s    in 26s     

2021-11-27 23:18:57 (22.2 MB/s) - ‘wsdm_data.tar.gz.4’ saved [613394184/613394184]

mkdir: cannot create directory ‘data’: File exists


# Первое задание
Собрать более-менее приличную рекомендательную систему с помощью GBDT. Использование трюков из лекции (и не из неё), тюнинг и разумных объемов фича инжиниринг прям нужен. 


Прочитаем данные:

In [2]:
import pandas as pd
members_data = pd.read_csv("data/members.csv")
songs_info_data = pd.read_csv("data/song_extra_info.csv")
songs_data = pd.read_csv("data/songs.csv")
train_data = pd.read_csv("data/train.csv")

Объединим их вместе в одну большую таблицу:

In [3]:
df = pd.merge(train_data, songs_data, on='song_id', how='left')
df = pd.merge(df, members_data, on='msno', how='left')
del members_data, songs_info_data, songs_data, train_data
df.head(2)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,20120102,20171005
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,NaN,NaN,52.0,13,24,female,9,20110525,20170911


Тут даты записаны как-то слитно, давайте подправим их на нормальный формат: 

In [4]:
df["registration_init_time"] = pd.to_datetime(df.registration_init_time, format='%Y%m%d', errors='ignore')
df["expiration_date"] = pd.to_datetime(df.expiration_date,  format='%Y%m%d', errors='ignore')
df.head(2)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language,city,bd,gender,registered_via,registration_init_time,expiration_date
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,1,0,NaN,7,2012-01-02,2017-10-05
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584.0,1259,Various Artists,NaN,NaN,52.0,13,24,female,9,2011-05-25,2017-09-11


Отлично, давайте определимся с категориальными фичами:


In [5]:
features = ['msno', 'song_id', 'source_screen_name', 'source_type', 'target',
       'artist_name', 'composer', 'registration_init_time']

df = df[['msno', 'song_id', 'source_screen_name', 'source_type', 'target',
       'song_length', 'artist_name', 'composer', 'bd', 'registration_init_time']]
df.head(2)

,msno,song_id,source_screen_name,source_type,target,song_length,artist_name,composer,bd,registration_init_time
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,Explore,online-playlist,1,206471.0,Bastille,Dan Smith| Mark Crew,0,2012-01-02
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,Local playlist more,local-playlist,1,284584.0,Various Artists,NaN,24,2011-05-25


Давайте превратим категориальные фичи в тип category, представим в виде вектора one_hot и запишем индексы:

In [6]:
for col in features:
    df[col] = df[col].astype('category').cat.codes
    df[col].fillna("unknown")

df.head(2)

,msno,song_id,source_screen_name,source_type,target,song_length,artist_name,composer,bd,registration_init_time
0,8158,74679,7,6,1,206471.0,3277,14581,0,2131
1,17259,223479,8,4,1,284584.0,31960,-1,24,1909


### Теперь обучим модель c помощью 5-fold и CatBoost. 

labels -- это target
data -- это все остальное, что мы оставили

In [7]:
labels = df['target']
df.pop('target')

data = df

Обучаем:

In [8]:
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold
import numpy as np

pred_labels = np.zeros(len(labels))

for train_id, test_id in KFold(n_splits=5).split(data, labels):
    train_data = df.iloc[train_id]
    train_labels = labels.iloc[train_id]

    model = CatBoostClassifier(learning_rate=0.1, depth=10, iterations=300, task_type="GPU")
    model.fit(train_data, train_labels)

    pred_labels[test_id] = model.predict_proba(data.iloc[test_id])[:, 1]

0:	learn: 0.6868637	total: 205ms	remaining: 1m 1s
1:	learn: 0.6817268	total: 388ms	remaining: 57.8s
2:	learn: 0.6775923	total: 551ms	remaining: 54.5s
3:	learn: 0.6742335	total: 729ms	remaining: 53.9s
4:	learn: 0.6714632	total: 891ms	remaining: 52.6s
5:	learn: 0.6692439	total: 1.05s	remaining: 51.6s
6:	learn: 0.6674327	total: 1.21s	remaining: 50.7s
7:	learn: 0.6659312	total: 1.37s	remaining: 50s
8:	learn: 0.6646797	total: 1.53s	remaining: 49.6s
9:	learn: 0.6636652	total: 1.69s	remaining: 49.1s
10:	learn: 0.6628146	total: 1.87s	remaining: 49.2s
11:	learn: 0.6620900	total: 2.04s	remaining: 49s
12:	learn: 0.6615069	total: 2.2s	remaining: 48.5s
13:	learn: 0.6610307	total: 2.35s	remaining: 48s
14:	learn: 0.6606044	total: 2.52s	remaining: 47.9s
15:	learn: 0.6601910	total: 2.71s	remaining: 48s
16:	learn: 0.6598034	total: 2.88s	remaining: 47.9s
17:	learn: 0.6594863	total: 3.06s	remaining: 47.9s
18:	learn: 0.6591893	total: 3.22s	remaining: 47.6s
19:	learn: 0.6589140	total: 3.38s	remaining: 47.3s

In [10]:
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

user_ids = np.unique(data["msno"])
aucs = []

for user_id in tqdm(user_ids):
  true_labels_of_cur_user = labels[user_id == data["msno"]]
  pred_labels_of_cur_user = pred_labels[user_id == data["msno"]]
  try:
    auc = roc_auc_score(true_labels_of_cur_user, pred_labels_of_cur_user )
  except ValueError:
    pass
  aucs.append(auc)

100%|██████████| 30755/30755 [05:01<00:00, 102.12it/s]


In [11]:
print("AUC: ", np.array(aucs).mean())

AUC:  0.6117545291427126


### Второе задание

Пользуясь информацией о треках построить неклассические эмбеддинги и попробовать использовать их:

а) показать симилары глазами
б) в качестве рекоммендера вместо GBDT (разницу в метриках) 
в) стек с GBDT (прирост / отсутствие / почему /, SHAP values)
